# MD2 in Rust

Today we're gonna go over writing the MD2 hash in rust.

The paper for this hash is [This RFC](https://www.rfc-editor.org/rfc/pdfrfc/rfc1319.txt.pdf)
And is not possible without the [errata](https://www.rfc-editor.org/rfc/inline-errata/rfc1319.html)

- First we import the library we need

In [39]:
use std::io::{self,Read};
extern crate hex;

Now we add the add the S Table, which uses the digits of PI as a sort of random number generator.

In [40]:
const S_TABLE: [u8; 256] = [
    0x29, 0x2E, 0x43, 0xC9, 0xA2, 0xD8, 0x7C, 0x01, 0x3D, 0x36, 0x54, 0xA1, 0xEC, 0xF0, 0x06, 0x13,
    0x62, 0xA7, 0x05, 0xF3, 0xC0, 0xC7, 0x73, 0x8C, 0x98, 0x93, 0x2B, 0xD9, 0xBC, 0x4C, 0x82, 0xCA,
    0x1E, 0x9B, 0x57, 0x3C, 0xFD, 0xD4, 0xE0, 0x16, 0x67, 0x42, 0x6F, 0x18, 0x8A, 0x17, 0xE5, 0x12,
    0xBE, 0x4E, 0xC4, 0xD6, 0xDA, 0x9E, 0xDE, 0x49, 0xA0, 0xFB, 0xF5, 0x8E, 0xBB, 0x2F, 0xEE, 0x7A,
    0xA9, 0x68, 0x79, 0x91, 0x15, 0xB2, 0x07, 0x3F, 0x94, 0xC2, 0x10, 0x89, 0x0B, 0x22, 0x5F, 0x21,
    0x80, 0x7F, 0x5D, 0x9A, 0x5A, 0x90, 0x32, 0x27, 0x35, 0x3E, 0xCC, 0xE7, 0xBF, 0xF7, 0x97, 0x03,
    0xFF, 0x19, 0x30, 0xB3, 0x48, 0xA5, 0xB5, 0xD1, 0xD7, 0x5E, 0x92, 0x2A, 0xAC, 0x56, 0xAA, 0xC6,
    0x4F, 0xB8, 0x38, 0xD2, 0x96, 0xA4, 0x7D, 0xB6, 0x76, 0xFC, 0x6B, 0xE2, 0x9C, 0x74, 0x04, 0xF1,
    0x45, 0x9D, 0x70, 0x59, 0x64, 0x71, 0x87, 0x20, 0x86, 0x5B, 0xCF, 0x65, 0xE6, 0x2D, 0xA8, 0x02,
    0x1B, 0x60, 0x25, 0xAD, 0xAE, 0xB0, 0xB9, 0xF6, 0x1C, 0x46, 0x61, 0x69, 0x34, 0x40, 0x7E, 0x0F,
    0x55, 0x47, 0xA3, 0x23, 0xDD, 0x51, 0xAF, 0x3A, 0xC3, 0x5C, 0xF9, 0xCE, 0xBA, 0xC5, 0xEA, 0x26,
    0x2C, 0x53, 0x0D, 0x6E, 0x85, 0x28, 0x84, 0x09, 0xD3, 0xDF, 0xCD, 0xF4, 0x41, 0x81, 0x4D, 0x52,
    0x6A, 0xDC, 0x37, 0xC8, 0x6C, 0xC1, 0xAB, 0xFA, 0x24, 0xE1, 0x7B, 0x08, 0x0C, 0xBD, 0xB1, 0x4A,
    0x78, 0x88, 0x95, 0x8B, 0xE3, 0x63, 0xE8, 0x6D, 0xE9, 0xCB, 0xD5, 0xFE, 0x3B, 0x00, 0x1D, 0x39,
    0xF2, 0xEF, 0xB7, 0x0E, 0x66, 0x58, 0xD0, 0xE4, 0xA6, 0x77, 0x72, 0xF8, 0xEB, 0x75, 0x4B, 0x0A,
    0x31, 0x44, 0x50, 0xB4, 0x8F, 0xED, 0x1F, 0x1A, 0xDB, 0x99, 0x8D, 0x33, 0x9F, 0x11, 0x83, 0x14,
];

First we initialize the data we will hash

In [41]:
let to_hash = "message digest".to_string().into_bytes();

Now let's make a mutable copy of this, and call it "data." We need this copy to be mutable.

In [42]:
let mut data = to_hash.clone();

### 3.1 Step 1. Append Padding Bytes

In the next step, we need to add padding bytes to the end of the data, so that the data hash a length that is a multiple of 16 bytes.

The RFC1319 requires that there be "*i* bytes of value *i*"

First we calculate the number of padding bytes required:

In [43]:
let padding_bytes = 16 - data.len() % 16;

Next, we form the vector that we will append to the data:

In [44]:
let mut padding = vec![padding_bytes as u8; padding_bytes];

Finally we append:

In [45]:
data.append(&mut padding);

### 3.2 Step 2. Append Checksum

We now add 16 checksum bytes to the end of the data that we have produced thus far.
The formula is outlined in pseudocode in RFC1319 as follows:

```
/* Clear checksum. */
For i = 0 to 15 do:
    Set C[i] to 0.
end /* of loop on i */

Set L to 0.

/* Process each 16-word block. */
For i = 0 to N/16-1 do

/* Checksum block i. */
    For j = 0 to 15 do
        Set c to M[i*16+j].
        INCORRECT, CORRECTED BY ERRATA: Set C[j] to S[c xor L].
        Set C[j] to C[j] xor S[c xor L].
        Set L to C[j].
    end /* of loop on j */
end /* of loop on i */
```

The array S referenced above is the S_TABLE we initialized earlier.

Let's begin by initializing a 16 bit vector with zeroes for our checksum, this takes care of the Clear checksum step. 

In [46]:
let mut cs = vec![0_u8; 16];

Set l to 0:

In [47]:
let mut l = 0_u8;

Let n be the length of our data:

In [48]:
let n = data.len();

Then, we will loop over each 16 word block in the data, performing the previously outlined computation.

In [49]:
for i in 0..(n / 16) {
    for j in 0..16 {
        let c = data[i * 16 + j];
        cs[j] ^= S_TABLE[(c ^ l) as usize];
        l = cs[j];
    }
}

()

Then, we append the checksum.

In [50]:
data.append(&mut cs);

### 3.3 Step 3. Initialize MD Buffer

The RFC reads "A 48-byte buffer X is used to compute the message digest. The buffer
is initialized to zero"

This is done simply:

In [51]:
let mut x = [0_u8;48];

### 3.4 Step 4. Process Message in 16-Byte Blocks

The Pseudocode reads as such:

```
/* Process each 16-word block. */
For i = 0 to N’/16-1 do

    /* Copy block i into X. */
    For j = 0 to 15 do
        Set X[16+j] to M[i*16+j].
        Set X[32+j] to (X[16+j] xor X[j]).
    end /* of loop on j */

    Set t to 0.
    /* Do 18 rounds. */
    For j = 0 to 17 do
        /* Round j. */
        For k = 0 to 47 do
            Set t and X[k] to (X[k] xor S[t]).
        end /* of loop on k */

        Set t to (t+j) modulo 256.
    end /* of loop on j */
end /* of loop on i */
```

This is done as so. I am also reinitializing the n variable for the new length.

In [52]:
let n = data.len();
for i in 0..(n / 16) {
    for j in 0..16 {
        x[16 + j] = data[i * 16 + j];
        x[32 + j] = x[16 + j] ^ x[j];
    }

    let mut t = 0_u8;

    for j in 0..18 {
        for k in 0..48 {
            x[k] ^= S_TABLE[t as usize];
            t = x[k];
        }
        t = ((t as u16 + j as u16) % 256) as u8;
    }
}

()

### 3.5 Step 5. Output

The mesage digest are the first sixteen bytes of the resulting array x. However, for readablility we will encode it to hex and return as a string.

In [53]:
{
    let digest = &x[0..16];
    let hash_str = hex::encode(digest);
    println!("{}",hash_str);
}

ab4f496bfb2a530b219ff33031fe06b0


()